In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
from skinet.utils.athena_tools import *
conn = athena_connection()
df = load_from_athena(conn, '''

    SELECT d.kunnr
	 , d.ctnum
     , a.cstat
     , a.gad_name
     , a.gad_start
     , a.cost_dept
     , a.conn_start
     , a.conn_end
     , CAST(a.monexp_amt AS DOUBLE)*100 AS monexp_amt
     , a.conn_type
     , a.ctype_l
     , a.ctype_m
     , a.itype_l
     , a.itype_m
     , a.itype_s
     , c.name1
     , a.first_bildate
     , c.telf2
     , a.gros_code
     , a.gross_type
     , a.kvgr4
     , b.sms_yn
     , b.email_yn
     , b.tm_yn
     , b.dm_yn
     , b.adt1_yn
     , b.aedat
     , b.aenam
  FROM sap.zsdt4013      AS a 
  LEFT JOIN (SELECT ctnum, kunnr, sms_yn, email_yn, tm_yn, dm_yn, sk1_yn, sk2_yn, adt1_yn, aedat, aenam, result1, result2, result3, result4, result5, result6 FROM sap.zsdt3748 WHERE usest = 'X') AS b
        ON a.ctnum = b.ctnum
  LEFT JOIN (SELECT kunnr, stcd2, stcd1, telf2, name1 FROM sap.kna1 WHERE mandt = '100') AS c
        ON a.kunnr = c.kunnr
LEFT JOIN (select distinct ctnum_hash, ctnum, kunnr_hash, kunnr from adt_work.hash_info) d
       ON a.ctnum = d.ctnum_hash
  WHERE a.mandt = '100' 
    AND a.stmon = '202009'  --년월 변경
    AND a.bukrs = '1000'
    AND a.cstat IN ('1','2')
    AND a.conn_type IN ('A01', 'A02')
    AND a.itype_m IN ('02', '05')
    AND b.adt1_yn IN ('Y', ' ') 
    AND b.sms_yn = 'Y'

''') 

...tmp_db name = tmp_20201019_11_34_05_30303ee5e1bc6b73e008c184029b7071b0d81720
Start creating table
drop table if exists default.tmp_20201019_11_34_05_30303ee5e1bc6b73e008c184029b7071b0d81720
<< table not exist

        create  table         default.tmp_20201019_11_34_05_30303ee5e1bc6b73e008c184029b7071b0d81720
        with (format='Parquet', parquet_compression='GZIP')
        as
        

    SELECT d.kunnr
	 , d.ctnum
     , a.cstat
     , a.gad_name
     , a.gad_start
     , a.cost_dept
     , a.conn_start
     , a.conn_end
     , CAST(a.monexp_amt AS DOUBLE)*100 AS monexp_amt
     , a.conn_type
     , a.ctype_l
     , a.ctype_m
     , a.itype_l
     , a.itype_m
     , a.itype_s
     , c.name1
     , a.first_bildate
     , c.telf2
     , a.gros_code
     , a.gross_type
     , a.kvgr4
     , b.sms_yn
     , b.email_yn
     , b.tm_yn
     , b.dm_yn
     , b.adt1_yn
     , b.aedat
     , b.aenam
  FROM sap.zsdt4013      AS a 
  LEFT JOIN (SELECT ctnum, kunnr, sms_yn, email_yn, tm_yn,

In [3]:
df.shape

(84594, 28)

In [4]:
def valid_number(x):
      nums = list(str(x)) 
      n = len(nums)
      answer=[]
      i = 3
      
      while i <= n-3:
            if int(nums[i]) - int(nums[i+1]) == int(nums[i+1]) - int(nums[i+2]):
                  answer.append(True)
            else:
                  answer.append(False)
            i += 1
      
      if (all(answer) == True) or (str(x) == '01000000001'):
            return False
      else:
            return True     

In [5]:
temp1 = df.copy()

In [6]:
temp1['telf2'] = temp1['telf2'].str.replace('-|/|' '|'  '', '', regex=True)
temp1['telf2'] = np.where(temp1['telf2'].str[0] != '0', "0"+temp1['telf2'], temp1['telf2'])
temp1['telf2'] = temp1['telf2'].str.replace(' ', '')
temp1['telf2'] = temp1['telf2'].str.replace('(', '')
temp1['telf2'] = temp1['telf2'].str.replace(')', '')
temp1['telf2'] = temp1['telf2'].str.replace('*', '')
temp1['telf2'] = temp1['telf2'].str.replace('B', '')
temp1['telf2'] = temp1['telf2'].str.replace('J', '')

In [7]:
temp1['telf_valid_1'] = temp1['telf2'].str.contains('^01([0|1|6|7|8|9]?)-?([0-9]{3,4})-?([0-9]{4})$', regex=True)
temp1['telf_valid_2'] = temp1['telf2'].apply(valid_number)
temp1['telf_valid_final'] = np.where((temp1['telf_valid_1'] == True) & (temp1['telf_valid_2'] == True), True, False)
temp1['telf_valid_final'] = temp1['telf_valid_final'].map({True : 1, False : 0})

In [8]:
temp1 = temp1[temp1['telf_valid_final'] == 1].drop_duplicates(subset='telf2', keep='first')

In [9]:
temp1.shape

(74228, 31)

In [10]:
temp1.columns

Index(['kunnr', 'ctnum', 'cstat', 'gad_name', 'gad_start', 'cost_dept',
       'conn_start', 'conn_end', 'monexp_amt', 'conn_type', 'ctype_l',
       'ctype_m', 'itype_l', 'itype_m', 'itype_s', 'name1', 'first_bildate',
       'telf2', 'gros_code', 'gross_type', 'kvgr4', 'sms_yn', 'email_yn',
       'tm_yn', 'dm_yn', 'adt1_yn', 'aedat', 'aenam', 'telf_valid_1',
       'telf_valid_2', 'telf_valid_final'],
      dtype='object')

In [12]:
final = temp1[[
    'kunnr', 'ctnum', 'gad_name', 'telf2', 'sms_yn', 'adt1_yn', 'itype_m', 'conn_type', 'monexp_amt', 'gad_start', 'cost_dept'
]]

final = final.rename(columns={
    'kunnr' : '고객번호',
    'ctnum' : '계약번호',
    'conn_type' : '계약타입',
    'itype_m' : '업종(중)',
    'gad_name' : '경비상호',
    'monexp_amt' : '월정료',
    'cost_dept' : '경비부서코드',
    'gad_start' : '경비개시일',
    'telf2' : '휴대폰번호',
    'sms_yn' : 'SMS수신동의',
    'adt1_yn' : '개인정보수집동의'
})

final = final[['고객번호', '계약번호', '업종(중)', '계약타입', '경비상호', '월정료', '경비부서코드', '경비개시일', '휴대폰번호', 'SMS수신동의', '개인정보수집동의']]
final.head()

,고객번호,계약번호,업종(중),계약타입,경비상호,월정료,경비부서코드,경비개시일,휴대폰번호,SMS수신동의,개인정보수집동의
0,CU0002288,100028271,05,A01,SK나이츠농구단정문,30000.0,571300,20170627,01037205413,Y,
2,CU0311543,217047955,05,A01,화진볼트산업,101000.0,711100,20170526,01038349608,Y,
3,CU1214209,217058159,02,A01,공차,60000.0,551600,20170629,01023555786,Y,
4,CU1216327,217062028,02,A01,족황상제 개포점,65000.0,511200,20170629,01088666676,Y,
5,CU1220115,217068552,02,A01,(주)류재은베이커리행복점,126000.0,521600,20170718,01054341383,Y,


In [13]:
final.to_csv('성장기획팀의뢰_SMS발송DB_201019.csv', encoding='utf-8-sig')